In [ ]:
# -*- coding: utf-8 -*-
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or
# implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# Demand prediction for bike sharing using LSTM 

**Based on:**
- https://github.com/curiousily/Deep-Learning-For-Hackers,
- https://curiousily.com/posts/demand-prediction-with-lstms-using-tensorflow-2-and-keras-in-python/

**Dataset:** https://www.kaggle.com/hmavrodiev/london-bike-sharing-dataset

---

1. **Trend Prediction in Deep Learning:**

   Trend prediction in deep learning refers to the process of analyzing historical data to forecast future trends. This process is critical in fields like finance, marketing, and meteorology, where understanding future trends can lead to better decision-making.

2. **Role of Time Series Data in Trend Prediction:**

   "Time Series Data" refers to a sequence of data points collected over time intervals. It's like taking snapshots of specific measurements at regular intervals. Think of it like recording the temperature every hour, or tracking the stock market’s closing price daily.

   *Example:* If you were to record the daily number of visitors to a website over a year, you'd have 365 data points, each representing a day.

   - **Characteristics of Time Series Data:**

    - *Temporal Dependence:* Each data point is related to its predecessors.
    - *Seasonality:* Regular patterns that repeat over time, like increased ice cream sales in summer.
    - *Trend:* Long-term increase or decrease in the data.

3. **Deep Learning for Trend Prediction:**

   Deep Learning models are highly suited for trend prediction due to their ability to process large volumes of data and learn complex patterns.

   - **Advantages:**
     - *Handling Non-linear Relationships:* They can capture complex, non-linear relationships in data.
     - *Feature Extraction:* Deep Learning models can automatically extract and learn important features from raw data, which is crucial in identifying underlying trends.

   - **Key Models Include:**

     - *Recurrent Neural Networks (RNNs):** Ideal for sequential data, they can remember past information and use it to influence current predictions.
     - *Long Short-Term Memory (LSTM) Networks:** A special kind of RNN, capable of learning long-term dependencies.
     - *Convolutional Neural Networks (CNNs):** Though primarily used for image data, they can also be effective for time series by treating sequences as one-dimensional images.

4. **LSTMs for Trend Prediction over Time Series Data:**

   As it was mentioned before, traditional neural networks struggle with ‘long-term dependencies’ – learning from data points that are far apart in time. LSTMs are designed to overcome this limitation.

   - **Key Features of LSTMs:**
     - *Memory Cells:* LSTMs have cells that can maintain information for long periods, essential for understanding long-term trends.
     - *Gates Mechanism:* They have a gate mechanism (input, output, and forget gates) that regulates the flow of information. These gates help the model to retain useful information and forget the irrelevant, enhancing its ability to predict trends.
     - *Robustness to Time Lags:* LSTMs are less sensitive to the gap length between important events in a time series, making them suitable for complex trend predictions.

5. **Example of LSTM in Trend Prediction:**

   Consider predicting the future trend of energy consumption in a city. An LSTM model can be trained on historical energy usage data, weather conditions, and other relevant variables collected over time. The LSTM would learn the underlying patterns, like increased energy usage in cold weather, and then use this knowledge to predict future energy trends.

6. **Challenges and Considerations:**

   - *Data Preprocessing:* Time Series data often requires careful preprocessing, such as normalization and handling missing values.
   - *Model Complexity:* LSTMs are complex and require substantial computational resources.
   - *Overfitting:* There's a risk of overfitting, especially with noisy or limited data.

In [ ]:
# !pip install gdown
# !pip install tensorflow-gpu

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from pandas.plotting import register_matplotlib_converters

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
!gdown --id 1nPw071R3tZi4zqVcmXA6kXVTe43Ex6K3 --output london_bike_sharing.csv

In [ ]:
df = pd.read_csv(
  "london_bike_sharing.csv",
  parse_dates=['timestamp'],
  index_col="timestamp"
)

In [ ]:
df.shape

Dataset features:    
  - **timestamp** - timestamp field for grouping the data
  - **cnt** - the count of a new bike shares
  - **t1** - real temperature in C
  - **t2** - temperature in C “feels like”
  - **hum** - humidity in percentage
  - **wind_speed** - wind speed in km/h
  - **weather_code** - category of the weather
  - **is_holiday** - boolean field - 1 holiday / 0 non holiday
  - **is_weekend** - boolean field - 1 if the day is weekend
  - **season** - category field meteorological seasons: 0-spring; 1-summer; 2-autumn; 3-winter.

In [ ]:
df.head()

In [ ]:
df['hour'] = df.index.hour
df['day_of_month'] = df.index.day
df['day_of_week'] = df.index.dayofweek
df['month'] = df.index.month

## Bike shares over time, diagram

In [ ]:
sns.lineplot(x=df.index, y="cnt", data=df);

In [ ]:
# Monthly

df_by_month = df.resample('M').sum()

sns.lineplot(x=df_by_month.index, y="cnt", data=df_by_month);

## Bike shares by the hour

In [ ]:
fig,(ax1, ax2, ax3, ax4)= plt.subplots(nrows=4)
fig.set_size_inches(18, 28)

sns.pointplot(data=df, x='hour', y='cnt', ax=ax1)
sns.pointplot(data=df, x='hour', y='cnt', hue='is_holiday', ax=ax2)
sns.pointplot(data=df, x='hour', y='cnt', hue='is_weekend', ax=ax3)
sns.pointplot(data=df, x='hour', y='cnt', hue='season', ax=ax4);

## Bike shares by the day of the week

In [ ]:
fig,(ax1, ax2)= plt.subplots(nrows=2)
fig.set_size_inches(18, 14)

sns.pointplot(data=df, x='day_of_week', y='cnt', ax=ax1)
sns.pointplot(data=df, x='day_of_week', y='cnt', hue='season', ax=ax2);

# Data preprocessing

In [ ]:
# 90% - train, 10% - test

train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

In [ ]:
# Scaling features in dataset

from sklearn.preprocessing import RobustScaler

f_columns = ['t1', 't2', 'hum', 'wind_speed']

f_transformer = RobustScaler()
cnt_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
cnt_transformer = cnt_transformer.fit(train[['cnt']])

train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
train['cnt'] = cnt_transformer.transform(train[['cnt']])

test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())
test['cnt'] = cnt_transformer.transform(test[['cnt']])

In [ ]:
# Creating a new dataset to prepare the sequences

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
# Each sequence is contains 10 data points from the observation history

time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.cnt, time_steps)
X_test, y_test = create_dataset(test, test.cnt, time_steps)

print(X_train.shape, y_train.shape)

# Trend Modeling: Simple Bidirectional LSTM with a Dropout layer

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
                                         
model = keras.Sequential()
model.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=128,
      input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    shuffle=False,
    callbacks=[callback]
)

# Evaluation

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend();

At about epoch 5, the model is already starting to overfit a bit. If you want, you can play around - regularize it, change the number of units, etc.

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_train_inv = cnt_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv = cnt_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = cnt_transformer.inverse_transform(y_pred)

### Visualization of ground truth vs prediction by our model

In [ ]:
# For all time

plt.plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv.flatten(), marker='.', label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv.flatten(), 'r', label="prediction")
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show();

In [ ]:
# In more visible form

plt.plot(y_test_inv.flatten()[:200], marker='.', label="true")
plt.plot(y_pred_inv.flatten()[:200], 'r', label="prediction")
plt.ylabel('Bike Count')
plt.xlabel('Time Step')
plt.legend()
plt.show();